# Preprocess CME Bitcoin Futures Price Data

In [11]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

# load data; The data is downloaded from BBT directly. 
xls = pd.ExcelFile('../data/btc future and reference rate/chains of btc future last price.xlsx')

# Contract Month Codes
month_code = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X','Z']
month = [1,2,3,4,5,6,7,8,9,10,11,12]
month_dict = dict(zip(month_code, month))

year = [2021, 2020, 2020, 2019, 2018]
year_code = ['1','0', '20', '19','18']
year_dict = dict(zip(year_code, year))

def last_Friday(y,m):
    day = dt.date(y,m,1)

    while day.weekday()!=4:
        day -= dt.timedelta(days=1)
    # Monday == 0 ... Sunday == 6
    last_Fri = day
    return last_Fri

df = pd.read_excel(xls, 'Sheet%s'%42)
future_name = df.iloc[2,1]
future_name = future_name.replace('BTC', '').replace(' Curncy', '')

y = year_dict[future_name[1:]]
m = month_dict[future_name[0]] 


In [12]:
# Concatenate BTC Futures prices into one time series
df_arr = []
future_name_arr = []
sheets = [43-i for i in range(1,43)]

for i in sheets:
    df = pd.read_excel(xls, 'Sheet%s'%i)
    
    # future name
    future_name = df.iloc[2,1]
    future_name = future_name.replace('BTC', '').replace(' Curncy', '')
    future_name_arr.append(future_name)

    y = year_dict[future_name[1:]]
    m = month_dict[future_name[0]] 

    lf = pd.to_datetime(last_Friday(y,m))
        
    df.columns = ['Date', 'PX_LAST']
    df = df.iloc[5:, :]
    df.reset_index(drop=True, inplace=True)
    df.Date = pd.to_datetime(df.Date)
    
    # remove price on and after expiry; time delta is to get some buffer
    df = df.loc[df.Date < lf - dt.timedelta(hours=1),:] 

    df.loc[:,'contract_name'] = ['BTC'+future_name+' Curncy']*len(df)
    df_arr.append(df)

In [13]:
# sort and drop duplicates
results = df_arr[0]

for df in df_arr[1:]:
    results = pd.concat([results, df])
    
results.drop_duplicates(subset=['Date'], keep='first', inplace=True)
results.sort_values('Date', ascending=False, inplace=True)
results.reset_index(inplace=True, drop=True)

# assign an hour entry to the daily closing price and convert to UTC; The CME BTC future close at 3pm US central time. 
results.Date = results.Date + pd.to_timedelta(15, unit='h')
results.index= results.Date
results.Date = results.index.tz_localize('US/Central').tz_convert('UTC')
results.reset_index(drop=True, inplace=True)

# save
# results.to_csv('../data/btc future and reference rate/concated_cleaned_bbt_future.csv')
# results.to_pickle('../data/btc future and reference rate/concated_cleaned_bbt_future.pkl')

In [34]:
# merge with hourly BTC prices from Tiingo
btc = pd.read_csv('../data/tiingo_btc.csv')
btc.date = pd.to_datetime(btc.date, utc = True)
btc = btc.loc[:,['date', 'open']]
btc.columns = ['Date', 'BTC Price']

merged = results.merge(btc, on = 'Date', how='inner')
merged = merged.drop_duplicates()
merged.PX_LAST = merged.PX_LAST.astype(np.float)

# log returns
merged.loc[:, 'log return future'] = np.log(merged.PX_LAST/merged.PX_LAST.shift(-1))
merged.loc[:, 'log return bitcoin'] = np.log(merged.loc[:, 'BTC Price']/merged.loc[:, 'BTC Price'].shift(-1))

# save
# merged.to_csv('../data/cleaned_data/BBT_Tiingo.csv')

# Bitwise related indices

In [ ]:
def convert_GMT(date):
    # convert date value from BITX data to utc time
    return pd.to_datetime(date[:-8]).tz_localize('US/Eastern').tz_convert('UTC')


# Data Slicing
We slice the cleaned data into training and testing sets.

In [1]:
import os

try:
    os.listdir('../../processed_data/%s/'%data_name)

except:
    print('create new folder')
    os.mkdir('../../processed_data/%s/'%data_name)
    os.mkdir('../../processed_data/%s/train'%data_name)
    os.mkdir('../../processed_data/%s/test'%data_name)

create new folder


NameError: name 'os' is not defined